In [19]:
# # https://www.tensorflow.org/ranking/tutorials/tfr_bert

# # import os
# # import tensorflow as tf

# #import tensorflow_ranking as tfr
# #from official.nlp.configs import encoders
# from tensorflow_ranking.extension.premade import tfrbert_task

# the last import got error ValueError: mutable default <class 'official.nlp.configs.encoders.EncoderConfig'> for field encoder is not allowed: use default_factory

from tensorflow_ranking.extension import pipeline

# Load package and pretrained model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


2024-03-17 23:59:43.547176: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 23:59:44.522485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

2024-03-17 23:59:47.022499: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you a

# Load data

In [124]:
# import pandas as pd
# # query
# query_train = pd.read_csv("./output/query_train_set_with_passage_info.csv")
# query_test =pd.read_csv("./output/query_test_set_with_passage_info.csv")

# # passage
# passage_train = pd.read_csv("./output/train_passage_id_content_cleaned.csv")
# passage_test = pd.read_csv("./output/test_passage_id_content_cleaned.csv")

In [37]:
passage_train = pd.DataFrame({
    'passage_id': [1, 2, 3, 4, 5, 6],
    'passage': ['I like bananas.', 'Bananas are my favorite fruit.', 'Apples are also good.', 'I prefer apples over bananas.', 'This is a passage about cooking.', 'Python is a programming language.']
})

# Example pandas DataFrame representing query_train
# Note: Replace this with your actual DataFrame
query_train = pd.DataFrame({
    'query_id': [1, 1, 1, 2, 2, 2],
    'query': ['What is the best fruit?', 'What is the best fruit?', 'What is the best fruit?', 'How to cook pasta?', 'How to cook pasta?', 'How to cook pasta?'],
    'passage_id': [1, 2, 3, 4, 5, 6],
    'rank': [1, 2, 3, 1, 2, 3],
    'score': [0.8, 0.7, 0.6, 0.9, 0.85, 0.75],
    'rel': [1, 0, 1, 1, 0, 1]
})

In [38]:
# Example pandas DataFrame representing query_test
# Note: Replace this with your actual DataFrame
query_test = pd.DataFrame({
    'query_id': [3, 3, 4, 4],
    'query': ['How to learn Spanish?', 'How to learn Spanish?', 'What is the capital of France?', 'What is the capital of France?'],
    'passage_id': [5, 6, 1, 2],
    'rank': [1, 2, 1, 2],
    'score': [0.85, 0.75, 0.8, 0.7],
    'rel': [1, 0, 1, 1]
})

# Example pandas DataFrame representing passage_test
# Note: Replace this with your actual DataFrame
passage_test = pd.DataFrame({
    'passage_id': [7, 8, 9, 10, 11, 12],
    'passage': ['Spanish is spoken in many countries.', 'Learning Spanish can be fun.', 'The capital of France is Paris.', 'Paris is a beautiful city.', 'Python is a programming language.', 'Python is widely used for data science.']
})


# (!! todo) Train (Use BERT and Fine tune)

In [73]:
# Function to compute BERT embeddings
def compute_embeddings(texts):
    tokenized_texts = tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='tf', max_length=32)
    embeddings = bert_model(tokenized_texts)[0][:, 0, :].numpy() # Use [CLS] token embedding
    return embeddings

In [75]:
passage_embeddings =  compute_embeddings(passage_train['passage'])

In [76]:
print(query_embeddings.shape)
print(passage_embeddings.shape)

(6, 768)
(6, 768)


In [77]:
# # Concatenate query and passage embeddings
concatenated_embeddings = tf.concat([query_embeddings, passage_embeddings], axis=1)

In [78]:


# Define dense layers
dense_layer_1 = Dense(256, activation='relu')(concatenated_embeddings)
dense_layer_2 = Dense(128, activation='relu')(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dense_layer_2)

# Define the model
model = Model(inputs=[query_input_ids, passage_input_ids], outputs=[output_layer])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ValueError: When providing `outputs` as a list/tuple, all values in the list/tuple must be KerasTensors. Received: outputs=[<tf.Tensor: shape=(6, 1), dtype=float32, numpy=
array([[0.5475777 ],
       [0.54350626],
       [0.5533128 ],
       [0.54824376],
       [0.56135833],
       [0.58291   ]], dtype=float32)>] including invalid value [[0.5475777 ]
 [0.54350626]
 [0.5533128 ]
 [0.54824376]
 [0.56135833]
 [0.58291   ]] of type <class 'tensorflow.python.framework.ops.EagerTensor'>

In [28]:


# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model (assuming you have labels for passage ranking)
# # Replace y_train with your actual labels
# model.fit(x=[query_inputs['input_ids'], passage_inputs['input_ids']], y=y_train, batch_size=32, epochs=5, validation_split=0.2)

# # Evaluate the model on test data
# # Replace y_test with your actual labels
# loss, accuracy = model.evaluate(x=[query_inputs_test['input_ids'], passage_inputs_test['input_ids']], y=y_test)
# print("Test Loss:", loss)
# print("Test Accuracy:", accuracy)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_1' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_model_1' (type TFBertModel):
  • input_ids=<KerasTensor shape=(None, 32), dtype=int32, sparse=None, name=query_input>
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None
  • output_hidden_states=None
  • return_dict=None
  • training=False

# Predict on Test Set (now using pretrained model)

In [50]:
passage_test = passage_train
query_test = query_train

In [51]:
# Function to compute BERT embeddings
def compute_embeddings(texts):
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=32)
    embeddings = bert_model(tokenized_texts)[0][:, 0, :].numpy().flatten()  # Use [CLS] token embedding
    return embeddings


In [52]:
# Tokenize queries and passages, and compute embeddings
passage_test["vector"] = passage_test["passage"].apply(lambda x :compute_embeddings(x))
query_test["vector"] = query_test["query"].apply(lambda x :compute_embeddings(x))

In [53]:
passage_test["vector"][3].shape

(768,)

In [54]:
len(passage_test["vector"][3])

768

In [55]:
query_test["vector"][3].shape

(768,)

In [56]:
query_test

,query_id,query,passage_id,rank,score,rel,vector
0,1,What is the best fruit?,1,1,0.80,1,"[-0.33224335, 0.23328725, -0.06571609, -0.0480..."
1,1,What is the best fruit?,2,2,0.70,0,"[-0.33224335, 0.23328725, -0.06571609, -0.0480..."
2,1,What is the best fruit?,3,3,0.60,1,"[-0.33224335, 0.23328725, -0.06571609, -0.0480..."
3,2,How to cook pasta?,4,1,0.90,1,"[-0.03376377, -0.13022594, -0.4259836, -0.0493..."
4,2,How to cook pasta?,5,2,0.85,0,"[-0.03376377, -0.13022594, -0.4259836, -0.0493..."
5,2,How to cook pasta?,6,3,0.75,1,"[-0.03376377, -0.13022594, -0.4259836, -0.0493..."


In [57]:

passage_test

,passage_id,passage,vector
0,1,I like bananas.,"[-0.061493658, 0.07960134, -0.3148743, -0.1612..."
1,2,Bananas are my favorite fruit.,"[0.009189999, 0.14451826, -0.4573639, -0.12262..."
2,3,Apples are also good.,"[-0.19710626, 0.10753094, -0.47878844, -0.1922..."
3,4,I prefer apples over bananas.,"[-0.071586266, 0.13607278, -0.5791471, -0.3332..."
4,5,This is a passage about cooking.,"[-0.30001095, 0.25777134, -0.67168224, -0.1257..."
5,6,Python is a programming language.,"[-0.21201693, -0.27617264, -0.31954288, -0.089..."


# Evaluate on Test Set

In [58]:
# Function for calculating average precision for a query
def average_precision(qid, qvector, topn = 10):  
    from sklearn.metrics.pairwise import cosine_similarity 
    # ========= for each query_id, do the following:
    #print(f"\nStarted to work on query_id {qid}")
    # get passage_id (each query_id would have upto 10 passage_id)
    qresult = query_test.loc[query_test["query_id"] == qid, ["passage_id", "rel"]]
    # get passage vector for each passage_id
    qcorpus = passage_test.loc[passage_test["passage_id"].isin(qresult["passage_id"]),["passage_id","vector"]]
    
    # inner join passage_id with passage vector
    # this is a result table of <= 10 rows, 2 columns: passage_id and vector
    qresult = pd.merge(qresult, qcorpus, on = "passage_id", how = "inner")

    # create a new column "similarity", 
    # by comparing similarity between the query vector (input qvector) with passage vector (the "vector" column)
    qresult["similarity"] = qresult["vector"].apply(lambda x: cosine_similarity(
                          np.array(qvector).reshape(1, -1)
                          , np.array(x).reshape(1, -1)).item())
    
    # sort rows so that passages with the highest similarity ranks as top 
    qresult.sort_values(by="similarity", ascending = False, inplace=True)

    # # Taking Top 10 documents for the evaluation
    ranking = qresult.head(topn)['rel'].values
    #display(qresult.head(15))
    
    # # Calculating precision
    precision=[]
    for i in range(1,len(ranking)+1):
      if ranking[i-1]:
        precision.append(np.sum(ranking[:i])/i)
    
    # If no relevant document in list then return 0
    if precision==[]:
      return 0
    #print(len(precision))
    #print(precision)
    avg_precision = np.mean(precision)
    #print("Avg precision: {}".format(avg_precision))
    return avg_precision

In [59]:
# Calculating average precision for all queries in the test set
query_test["avg_precision"] = query_test.apply(lambda x: average_precision(x["query_id"]
                                                                           ,x["vector"]),axis=1)

In [60]:
query_test

,query_id,query,passage_id,rank,score,rel,vector,avg_precision
0,1,What is the best fruit?,1,1,0.80,1,"[-0.33224335, 0.23328725, -0.06571609, -0.0480...",0.583333
1,1,What is the best fruit?,2,2,0.70,0,"[-0.33224335, 0.23328725, -0.06571609, -0.0480...",0.583333
2,1,What is the best fruit?,3,3,0.60,1,"[-0.33224335, 0.23328725, -0.06571609, -0.0480...",0.583333
3,2,How to cook pasta?,4,1,0.90,1,"[-0.03376377, -0.13022594, -0.4259836, -0.0493...",0.833333
4,2,How to cook pasta?,5,2,0.85,0,"[-0.03376377, -0.13022594, -0.4259836, -0.0493...",0.833333
5,2,How to cook pasta?,6,3,0.75,1,"[-0.03376377, -0.13022594, -0.4259836, -0.0493...",0.833333


In [61]:
print('Mean Average Precision=>', query_test["avg_precision"].mean())

Mean Average Precision=> 0.7083333333333331
